In [1]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

df = web.DataReader('AAPL', data_source='yahoo',
                    start='2012-01-01', end='2021-06-04')

In [ ]:
# show the table
# print(df)

# show the rows and columns
# print(df.shape)

plt.figure(figsize=(16, 8))
plt.title('Close price history')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.show()

# create a new dataframe with only the 'close' column
data = df.filter(['Close'])

# convert the dataframe into a numpy array
dataset = data.values